In [5]:
import os
import warnings
from pathlib import Path

#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import seaborn as sns
from IPython.display import display
from pandas.api.types import CategoricalDtype

from category_encoders import MEstimateEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import KFold, cross_val_score
#from xgboost import XGBRegressor

'''
# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)
'''
# Mute warnings
warnings.filterwarnings('ignore')

In [6]:
df = pd.read_csv("newDataFrame.csv", error_bad_lines=False)
df

,date,houseId,city,address,houseType,status,numberRooms,numberBathrooms1,numberBathrooms,numberKitchens1,...,landArea,elevator1,elevator,carParking1,carParking,heating1,heating,price,priceType,priceUS
0,2020-01-29,240003,RB,- بيتونيا,apartment,A,1,0,NaN,0,...,NaN,1,نعم,1,نعم,0,NaN,1300,شيكل,390
1,2020-01-29,240005,RB,- مدينة رام الله,apartment,A,1,1,1,1,...,NaN,1,نعم,1,نعم,0,NaN,550,دولار,550
2,2020-01-29,240006,RB,- مدينة رام الله,apartment,A,3,3,3,0,...,100.0,1,نعم,1,نعم,1,نعم,1000,دولار,1000
3,2020-01-29,240007,RB,- مدينة رام الله,apartment,A,3,3,3,1,...,NaN,1,نعم,1,نعم,1,نعم,950,دولار,950
4,2020-01-29,240008,RB,- مدينة رام الله,apartment,A,3,3,3,0,...,NaN,1,نعم,1,نعم,0,NaN,800,دولار,800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8156,2021-03-03,260069,RB,- مدينة البيرة,apartment,A,2,2,2,1,...,NaN,1,نعم,1,نعم,0,NaN,600,دولار,600
8157,2021-03-03,260076,RB,- مدينة البيرة,apartment,B,3,2,2,1,...,NaN,1,نعم,1,نعم,0,NaN,500,دولار,500
8158,2021-03-03,260079,H,- مدينة الخليل,apartment,B,3,2,2,1,...,NaN,0,NaN,0,NaN,0,NaN,1600,شيكل,480
8159,2021-03-03,260080,N,- مدينة نابلس,apartment,A,1,1,1,1,...,3500.0,1,نعم,1,نعم,1,نعم,1000,شيكل,300


In [7]:
df.info

<bound method DataFrame.info of             date  houseId city            address  houseType status  \
0     2020-01-29   240003   RB          - بيتونيا  apartment      A   
1     2020-01-29   240005   RB  -  مدينة رام الله  apartment      A   
2     2020-01-29   240006   RB  -  مدينة رام الله  apartment      A   
3     2020-01-29   240007   RB  -  مدينة رام الله  apartment      A   
4     2020-01-29   240008   RB  -  مدينة رام الله  apartment      A   
...          ...      ...  ...                ...        ...    ...   
8156  2021-03-03   260069   RB    -  مدينة البيرة  apartment      A   
8157  2021-03-03   260076   RB    -  مدينة البيرة  apartment      B   
8158  2021-03-03   260079    H    -  مدينة الخليل  apartment      B   
8159  2021-03-03   260080    N     -  مدينة نابلس  apartment      A   
8160  2021-03-03   260085   RB    -  مدينة البيرة  apartment      A   

      numberRooms numberBathrooms1 numberBathrooms numberKitchens1  ...  \
0               1                0      

In [9]:
#Handling Missing Values
# get the number of missing data points per column
missing_values = df.isnull().sum()
# look at the # of missing points in the first 25 columns
missing_values[0:25]

date                   0
houseId                0
city                   0
address               55
houseType             17
status                 0
numberRooms            0
numberBathrooms1       0
numberBathrooms      272
numberKitchens1        0
numberKitchens       364
numberBalconies1       0
numberBalconies     1322
loungeType           851
houseSpace          2085
landArea            5709
elevator1              0
elevator            3273
carParking1            0
carParking          2828
heating1               0
heating             6350
price                  0
priceType              1
priceUS                0
dtype: int64

In [10]:
#total missing values do we have
total_cells = np.product(df.shape)
total_missing = missing_values.sum()
# percent of data that is missing
percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

11.335375566719765


In [11]:
total_missing

23127

In [12]:
total_cells

204025

In [13]:
# replace all NA's the value that comes directly after it in the same column,
numberBathroomsNew = df.numberBathrooms
df.numberBathrooms = numberBathroomsNew.fillna(method='bfill', axis=0)
df.numberBathrooms.isnull().sum()


0

In [14]:
houseTypeNew = df.houseType
df.houseType = houseTypeNew.fillna(method='bfill')
df.houseType.isnull().sum()

0

In [15]:
addressNew = df.address
df.address = addressNew.fillna(method='bfill')
df.address.isnull().sum()

0

In [16]:
numberKitchensNew = df.numberKitchens
df.numberKitchens = numberKitchensNew.fillna(method='bfill')
df.numberKitchens.isnull().sum()

0

In [17]:
# replace all NA's with 0
numberBalconiesNew = df.numberBalconies
df.numberBalconies = numberBalconiesNew.fillna(0)
df.numberBalconies.isnull().sum()

0

In [18]:
df.elevator = df.elevator1
df.carParking = df.carParking1
df.heating = df.heating1


In [19]:
missing_values = df.isnull().sum()
missing_values[0:25]

date                   0
houseId                0
city                   0
address                0
houseType              0
status                 0
numberRooms            0
numberBathrooms1       0
numberBathrooms        0
numberKitchens1        0
numberKitchens         0
numberBalconies1       0
numberBalconies        0
loungeType           851
houseSpace          2085
landArea            5709
elevator1              0
elevator               0
carParking1            0
carParking             0
heating1               0
heating                0
price                  0
priceType              1
priceUS                0
dtype: int64

In [20]:
# get all the unique values in the 'loungeType , city ' column
lounge = df['loungeType'].unique()


In [21]:
#city
city = df['priceUS'].unique()


In [24]:
#drop extra columns
newdf = df.drop(['houseId', 'address', 'numberBathrooms1','numberKitchens1','numberBalconies1','loungeType','landArea','elevator1','carParking1','heating1','priceType','price'], axis=1)
newdf

,date,city,houseType,status,numberRooms,numberBathrooms,numberKitchens,numberBalconies,houseSpace,elevator,carParking,heating,priceUS
0,2020-01-29,RB,apartment,A,1,1,1,0,NaN,1,1,0,390
1,2020-01-29,RB,apartment,A,1,1,1,1,80,1,1,0,550
2,2020-01-29,RB,apartment,A,3,3,1,2,190,1,1,1,1000
3,2020-01-29,RB,apartment,A,3,3,1,1,190,1,1,1,950
4,2020-01-29,RB,apartment,A,3,3,1,3,160,1,1,0,800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8156,2021-03-03,RB,apartment,A,2,2,1,1,120,1,1,0,600
8157,2021-03-03,RB,apartment,B,3,2,1,1,140,1,1,0,500
8158,2021-03-03,H,apartment,B,3,2,1,2,150,0,0,0,480
8159,2021-03-03,N,apartment,A,1,1,1,1,70,1,1,1,300


In [25]:
newdf = newdf.drop(['priceUS','houseSpace'], axis=1)
newdf

,date,city,houseType,status,numberRooms,numberBathrooms,numberKitchens,numberBalconies,elevator,carParking,heating
0,2020-01-29,RB,apartment,A,1,1,1,0,1,1,0
1,2020-01-29,RB,apartment,A,1,1,1,1,1,1,0
2,2020-01-29,RB,apartment,A,3,3,1,2,1,1,1
3,2020-01-29,RB,apartment,A,3,3,1,1,1,1,1
4,2020-01-29,RB,apartment,A,3,3,1,3,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
8156,2021-03-03,RB,apartment,A,2,2,1,1,1,1,0
8157,2021-03-03,RB,apartment,B,3,2,1,1,1,1,0
8158,2021-03-03,H,apartment,B,3,2,1,2,0,0,0
8159,2021-03-03,N,apartment,A,1,1,1,1,1,1,1


In [27]:
ph = pd.read_csv("PriceType_houseSpace.csv", error_bad_lines=False)
ph = ph.drop(['Unnamed: 0'], axis=1)
ph

,houseSpace,PriceType_JOD,PriceType_NIS,PriceType_USD,price
0,140.0,0,1,0,1300
1,80.0,0,0,1,550
2,190.0,0,0,1,1000
3,190.0,0,0,1,950
4,160.0,0,0,1,800
...,...,...,...,...,...
8156,120.0,0,0,1,600
8157,140.0,0,0,1,500
8158,150.0,0,1,0,1600
8159,70.0,0,1,0,1000


In [28]:
newdf = pd.concat([newdf, ph], axis=1)
newdf

,date,city,houseType,status,numberRooms,numberBathrooms,numberKitchens,numberBalconies,elevator,carParking,heating,houseSpace,PriceType_JOD,PriceType_NIS,PriceType_USD,price
0,2020-01-29,RB,apartment,A,1,1,1,0,1,1,0,140.0,0,1,0,1300
1,2020-01-29,RB,apartment,A,1,1,1,1,1,1,0,80.0,0,0,1,550
2,2020-01-29,RB,apartment,A,3,3,1,2,1,1,1,190.0,0,0,1,1000
3,2020-01-29,RB,apartment,A,3,3,1,1,1,1,1,190.0,0,0,1,950
4,2020-01-29,RB,apartment,A,3,3,1,3,1,1,0,160.0,0,0,1,800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8156,2021-03-03,RB,apartment,A,2,2,1,1,1,1,0,120.0,0,0,1,600
8157,2021-03-03,RB,apartment,B,3,2,1,1,1,1,0,140.0,0,0,1,500
8158,2021-03-03,H,apartment,B,3,2,1,2,0,0,0,150.0,0,1,0,1600
8159,2021-03-03,N,apartment,A,1,1,1,1,1,1,1,70.0,0,1,0,1000


In [33]:
missing_values = newdf.isnull().sum()
missing_values[0:16]

date               0
city               0
houseType          0
status             0
numberRooms        0
numberBathrooms    0
numberKitchens     0
numberBalconies    0
elevator           0
carParking         0
heating            0
houseSpace         0
PriceType_JOD      0
PriceType_NIS      0
PriceType_USD      0
price              0
dtype: int64

In [34]:
newdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8161 entries, 0 to 8160
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             8161 non-null   object 
 1   city             8161 non-null   object 
 2   houseType        8161 non-null   object 
 3   status           8161 non-null   object 
 4   numberRooms      8161 non-null   int64  
 5   numberBathrooms  8161 non-null   object 
 6   numberKitchens   8161 non-null   object 
 7   numberBalconies  8161 non-null   object 
 8   elevator         8161 non-null   int64  
 9   carParking       8161 non-null   int64  
 10  heating          8161 non-null   int64  
 11  houseSpace       8161 non-null   float64
 12  PriceType_JOD    8161 non-null   int64  
 13  PriceType_NIS    8161 non-null   int64  
 14  PriceType_USD    8161 non-null   int64  
 15  price            8161 non-null   object 
dtypes: float64(1), int64(7), object(8)
memory usage: 1020.2+ KB


In [35]:
#Handling unique data
uniqueBathrooms = newdf['numberBathrooms'].unique()
uniqueBathrooms

array(['1', '3', '2', '-2', '-1', '-3', '-4', '2 حمام', '4', '0', '5',
       '6', 'حمامين'], dtype=object)

In [36]:
#Handling unique data
newdf['numberBathrooms'] = newdf['numberBathrooms'].replace(['حمامين'],'2')
newdf['numberBathrooms'] = newdf['numberBathrooms'].replace(['-1'],'1')
newdf['numberBathrooms'] = newdf['numberBathrooms'].replace(['-2'],'2')
newdf['numberBathrooms'] = newdf['numberBathrooms'].replace(['-3'],'3')
newdf['numberBathrooms'] = newdf['numberBathrooms'].replace(['-4'],'4')
newdf['numberBathrooms'] = newdf['numberBathrooms'].replace(['حمام'],'1')

In [37]:
uniqueBalconies = newdf['numberBalconies'].unique()
uniqueBalconies

array([0, '1', '2', '3', '4', '0', '-2', '-1', '5', '2 برنده', '120',
       'واجده', '100', '12', '21', '150', '180', '6', '14', '-3'],
      dtype=object)

In [38]:
newdf['numberKitchens'] = newdf['numberKitchens'].replace(['واجد'],'1')

In [39]:
newdf['numberBalconies'] = newdf['numberBalconies'].replace(['واجده'],'1')

In [40]:
newdf['numberBalconies'] = newdf['numberBalconies'].replace(['-2'],'2')

In [41]:
newdf['numberBalconies'] = newdf['numberBalconies'].replace(['-1'],'1')

In [42]:
newdf['numberBalconies'] = newdf['numberBalconies'].replace(['-3'],'3')

array([ 1.4000e+02,  8.0000e+01,  1.9000e+02,  1.6000e+02,  1.5000e+02,
        2.0000e+02,  1.3000e+02,  1.7500e+02,  1.3500e+02,  2.1000e+02,
        1.2000e+02,  2.2000e+02,  9.0000e+01,  1.1900e+02,  1.2500e+02,
        1.7400e+02,  1.4500e+02,  5.0000e+01,  7.0000e+01,  1.9500e+02,
        1.1600e+02,  1.1000e+02,  1.6500e+02,  1.8000e+02,  1.7000e+02,
        1.5500e+02,  2.0000e+01,  1.8500e+02,  1.0000e+02,  2.5000e+01,
        1.1500e+02,  3.0000e+02,  0.0000e+00,  2.8000e+02,  4.5000e+01,
        1.2400e+02,  1.2700e+02,  1.4400e+02,  1.6200e+02,  1.2300e+02,
        1.6000e+01,  1.7200e+02,  1.4800e+02,  1.2200e+02,  8.8000e+01,
        1.1800e+02,  6.0000e+01,  1.3800e+02,  1.0000e+00,  8.5000e+01,
        4.0000e+01,  2.6000e+01,  2.1500e+02,  1.6600e+02,  1.6800e+02,
        1.0000e+03,  1.3200e+02,  2.6000e+02,  2.7000e+02,  3.0000e+01,
        7.5000e+01,  1.3900e+02,  1.6900e+02,  2.5000e+02, -1.4000e+02,
        3.5000e+02,  1.8700e+02,  1.1500e+03,  1.0500e+02,  1.17

In [45]:
uniquehouseType = newdf['houseType'].unique()
uniquehouseType

array(['apartment', 'Roof', 'villa'], dtype=object)

In [46]:
newdf.to_csv (r'df.csv', index = False, header=True)

In [70]:
df = pd.read_csv("df.csv", error_bad_lines=False)
df

,date,city,houseType,status,numberRooms,numberBathrooms,numberKitchens,numberBalconies,elevator,carParking,heating,houseSpace,PriceType_JOD,PriceType_NIS,PriceType_USD,price
0,2020-01-29,RB,apartment,A,1,1,1,0,1,1,0,140.0,0,1,0,1300
1,2020-01-29,RB,apartment,A,1,1,1,1,1,1,0,80.0,0,0,1,550
2,2020-01-29,RB,apartment,A,3,3,1,2,1,1,1,190.0,0,0,1,1000
3,2020-01-29,RB,apartment,A,3,3,1,1,1,1,1,190.0,0,0,1,950
4,2020-01-29,RB,apartment,A,3,3,1,3,1,1,0,160.0,0,0,1,800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8152,2021-03-03,RB,apartment,A,2,2,1,1,1,1,0,120.0,0,0,1,600
8153,2021-03-03,RB,apartment,B,3,2,1,1,1,1,0,140.0,0,0,1,500
8154,2021-03-03,H,apartment,B,3,2,1,2,0,0,0,150.0,0,1,0,1600
8155,2021-03-03,N,apartment,A,1,1,1,1,1,1,1,70.0,0,1,0,1000


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8157 entries, 0 to 8156
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             8157 non-null   object 
 1   city             8157 non-null   object 
 2   houseType        8157 non-null   object 
 3   status           8157 non-null   object 
 4   numberRooms      8157 non-null   int64  
 5   numberBathrooms  8157 non-null   int64  
 6   numberKitchens   8157 non-null   int64  
 7   numberBalconies  8157 non-null   int64  
 8   elevator         8157 non-null   int64  
 9   carParking       8157 non-null   int64  
 10  heating          8157 non-null   int64  
 11  houseSpace       8157 non-null   float64
 12  PriceType_JOD    8157 non-null   int64  
 13  PriceType_NIS    8157 non-null   int64  
 14  PriceType_USD    8157 non-null   int64  
 15  price            8157 non-null   int64  
dtypes: float64(1), int64(11), object(4)
memory usage: 1019.8+ KB

In [72]:
df['heating'] = df['heating'].astype(int) 
df['carParking'] = df['carParking'].astype(int)  
# displaying the datatypes 
display(df.dtypes) 

date                object
city                object
houseType           object
status              object
numberRooms          int64
numberBathrooms      int64
numberKitchens       int64
numberBalconies      int64
elevator             int64
carParking           int32
heating              int32
houseSpace         float64
PriceType_JOD        int64
PriceType_NIS        int64
PriceType_USD        int64
price                int64
dtype: object

In [73]:
uniquenumberBathrooms = df['numberBathrooms'].unique()
uniquenumberBathrooms

array([1, 3, 2, 4, 0, 5, 6], dtype=int64)

In [74]:
uniquenumberBalconies = df['price'].unique()
uniquenumberBalconies

array([   1300,     550,    1000,     950,     800,     450,     700,
           600,    2300,    1100,    2700,     900,    2000,     500,
           400,    2200,     850,    1600,     320,    1800,     200,
          3000,    1200,    3300,     350,     430,  160000,   12000,
           650,    1500,    1400,    1900,    1550,     750,    2500,
           300,     280,  180000,    1450,    4000,    2400,     180,
           425,    2800,    3500,    2600,    1650,     330,     340,
          1250,   15000,    2100,    1150,    5009,    1700,     250,
        140000,  310000,    3700,     170,   13200,    1750,   50000,
           100,  235000,   10800,    3400,     125,  250000,    3200,
           370,   70000,    1350,    5000,    3600,    2999,    6000,
           680,     230,     660,    1670,     310,     480,     160,
           260,  115000,  175000,      50,    7000,    3800,    2900,
         80000,     290,   60000,      40,       1,     140,   18000,
         20000,  190

In [75]:
df.describe()

,numberRooms,numberBathrooms,numberKitchens,numberBalconies,elevator,carParking,heating,houseSpace,PriceType_JOD,PriceType_NIS,PriceType_USD,price
count,8157.000000,8157.000000,8157.000000,8157.000000,8157.000000,8157.000000,8157.000000,8157.000000,8157.000000,8157.000000,8157.000000,8.157000e+03
mean,2.495893,1.865514,1.033591,1.096236,0.598750,0.653794,0.222018,180.989947,0.100405,0.410690,0.488783,3.029254e+03
std,0.871115,0.749323,1.669212,3.196358,0.490182,0.475789,0.415629,2814.771157,0.300557,0.491989,0.499905,2.847031e+04
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-140.000000,0.000000,0.000000,0.000000,1.000000e+00
25%,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,120.000000,0.000000,0.000000,0.000000,5.000000e+02
50%,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,0.000000,140.000000,0.000000,0.000000,0.000000,8.000000e+02
75%,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,0.000000,150.000000,0.000000,1.000000,1.000000,1.400000e+03
max,6.000000,6.000000,150.000000,180.000000,1.000000,1.000000,1.000000,190000.000000,1.000000,1.000000,1.000000,1.400000e+06


In [76]:
index_names = df[ df['numberBalconies'] > 5 ].index 
index_names
# drop these row indexes 
# from dataFrame 
df.drop(index_names, inplace = True) 


In [77]:
index_names = df[ df['numberBathrooms'] > 4 ].index 
index_names
# drop these row indexes 
# from dataFrame 
df.drop(index_names, inplace = True) 


In [78]:
index_names = df[ df['numberKitchens'] > 3 ].index 
index_names
# drop these row indexes 
# from dataFrame 
df.drop(index_names, inplace = True) 

In [79]:
df.to_csv (r'dfC.csv', index = False, header=True)